# Computer Vision 1 - Assignment 3 - Problem 2
----
Computing homography for two cameras via RANSAC.

Stitch panorama picture from two images subsequently.

## Setup of working directory and loading of necessary packages

In [5]:
if pwd() != "C:\\Users\\Max\\Documents\\jupyter-notebooks\\Julia"
    cd("C:\\Users\\Max\\Documents\\jupyter-notebooks\\Julia")
end
println(pwd())
using Images
using PyPlot
using Grid
using JLD

C:\Users\Max\Documents\jupyter-notebooks\Julia


## Loads keypoints from JLD container

* **Inputs**
    
    filename JLD container filename
    
    
* **Outputs**
    
    keypoints1 [n x 2] keypoint locations (of left image)
    
    keypoints2 [m x 2] keypoint locations (of right image)
    

In [6]:
function loadkeypoints(filename::String) 
    keypoints1 = JLD.load(filename,"keypoints1");
    keypoints2 = JLD.load(filename,"keypoints2");
    
    @assert size(keypoints1,2) == 2
    @assert size(keypoints2,2) == 2
    return keypoints1::Array{Int64,2}, keypoints2::Array{Int64,2}
end

loadkeypoints (generic function with 1 method)

## Compute chi-squared matrix
----
Computes pairwise euclidean distance for all keypoint pairs

Dimension $D$ of feature vectors here equal to $128$

----
* **Inputs**

    features1 [128 x n] descriptors of first image  (denoted $p$) 
    
    features2 [128 x m] descriptors of second image (denoted $q$)
----
* **Outputs**

    D [n x m] distance matrix.
    
    Entries of matrix $D$ are pairwise euclidean squared distance:
    
    $$D_{ij} = \sum_{k=1}^{D}\left(q_{jk}-p_{ik}\right)^2$$

In [9]:
function euclideansquaredist(features1::Array{Float64,2},features2::Array{Float64,2})
    
    dimLeft  = size(features1,2);
    dimRight = size(features2,2);
    D = zeros(dimLeft,dimRight);
    for i=1:dimLeft 
        for j=1:dimRight
            D[i,j] = sum((features2[:,j]-features1[:,i]).^2);
        end
    end

    @assert size(D) == (size(features1,2),size(features2,2))
    return D::Array{Float64,2}
end

euclideansquaredist (generic function with 1 method)

, Array{Float64, 2}) in module Main at In[8]:3 overwritten at In[9]:3.


## Match keypoints from left image to keypoints from right image using the distance matrix
----
* **Inputs**

    p1 [n x 2] keypoints coordinates in first image
    
    p2 [m x 2] keypoints coordinates in second image
    
    D  [n x m] distance matrix 
----
* **Outputs**

    pairs [min(n,m) x 4] vector such that each row holds the coordinates of an interest point in p1 and p2

In [10]:
function findmatches(p1::Array{Int,2}, p2::Array{Int,2}, D::Array{Float64,2})
    if size(p1,1) > size(p2,1)
        larger = p1;
        smaller = p2;
    else
        larger = p2;
        smaller = p1;
    end
    pairs = zeros(Int,size(smaller,1),4)
    for i=1:size(smaller,1)
        pairs[i,1:2] = smaller[i,:];
        pairs[i,3:4] = larger[indmin(D[i,:]),:]
    end
    
    @assert size(pairs) == (min( size(p1,1),size(p2,1)),4)
    return pairs::Array{Int,2}
end

findmatches (generic function with 1 method)

## Data & Parameters:

----

* Parameters

    * $\sigma$ := standard deviation for presmoothing derivatives, used by SIFT    

    * ransac_threshold := maximum distance for a keypoint to be considered an inlier

    * p := Probability that any given keypoint correspondence is valid

    * k := Number of samples drawn per RANSAC iteration, used to compute homography

    * z := Total probability of success after all RANSAC iterations

----

* Data

    * im1,im2 := respective Image from either camera

    * kp1,kp2 := keypoints detected in either image
    
    * features1, features2 := Features detected by SIFT algorithm

In [11]:
include("Common.jl")
# Parameters:
sigma = 1.4;
ransac_threshold = 50.0;
p=0.5;
k=4;
z=0.99;
# Data
im1 = Float64.(PyPlot.imread("a3p2a.png"));
im2 = Float64.(PyPlot.imread("a3p2b.png"));
println(pwd())
kp1,kp2 = loadkeypoints("keypoints.jld");
features1 = Common.sift(kp1, im1, sigma);
features2 = Common.sift(kp2, im2, sigma);


C:\Users\Max\Documents\jupyter-notebooks\Julia


## RANSAC algorithm to estimate homography

In [13]:
# compute chi-square distance  matrix
D = euclideansquaredist(features1,features2)
# find matching pairs
pairs = findmatches(kp1,kp2,D)
# show matches
#showmatches(im1,im2,pairs)
#title("Putative Matching Pairs")
# compute number of iterations for the RANSAC algorithm
# niterations = computeransaciterations(p,k,z)
# apply RANSAC
# bestinliers,bestpairs,bestH = ransac(pairs,ransac_threshold,niterations)
test = rand(10,10)

10×10 Array{Float64,2}:
 0.837253  0.774586  0.552019   0.00616342  …  0.333802  0.149125  0.329312 
 0.821227  0.63416   0.10211    0.820734       0.122835  0.456332  0.0508534
 0.652726  0.369218  0.64227    0.664721       0.352634  0.169936  0.651129 
 0.327661  0.164057  0.113532   0.341892       0.83664   0.100279  0.932827 
 0.902292  0.126359  0.859571   0.0478882      0.622509  0.697004  0.700652 
 0.114202  0.577459  0.989234   0.739662    …  0.728106  0.745997  0.680774 
 0.360921  0.609849  0.919518   0.197528       0.654225  0.699731  0.169788 
 0.474905  0.216183  0.0213359  0.216482       0.139892  0.406547  0.378065 
 0.143901  0.8927    0.25097    0.860465       0.367379  0.696621  0.660413 
 0.163213  0.953219  0.741503   0.887771       0.497133  0.834335  0.372268 

In [16]:
@show size(pairs)
@show size(kp1)
@show size(kp2)

size(pairs) = (123,4)
size(kp1) = (123,2)
size(kp2) = (140,2)


(140,2)